In [14]:
import pandas as pd
import pyreadr
import json
import os
import re
from datetime import date
import datetime

In [2]:
os.chdir('/Users/cblanesg/misinformation_socialmedia/data')
os.getcwd()

'/Users/cblanesg/misinformation_socialmedia/data'

## Functions

In [3]:
def obtain_newspapers_id():
    result = pyreadr.read_r('5-analysis/1-input_data/misinformation/data_reg_misinformation.Rda')
    input_reg = result["input_reg"]
    df = input_reg[input_reg['type_misinformation'] == 'media_outlets'][['id_desinformacion', 
                                                               'id_factcheck', 
                                                               'link_desinformacion', 
                                                               'label_desinformacion']].drop_duplicates()
    
    df['media_outlet'] = df['link_desinformacion'].apply(lambda x: re.sub('https://|www.|http://', '',x).split('.')[0])
    return(df)

In [4]:
def obtain_id_newspapers():
    ids = pd.read_excel('3-synthetic_controls/newspapers/0-ids/controls_id.xlsx')[['id_desinformacion', 
                                                                                   'link_desinformacion', 
                                                                                   'link_medio', 
                                                                                   'date_desinformacion']]
    
    ids_control = pd.read_excel('3-synthetic_controls/newspapers/0-ids/controls_true_id.xlsx')[['id_desinformacion', 
                                                                                   'link_desinformacion', 
                                                                                   'link_medio', 
                                                                                   'date_desinformacion']]
    
    id_all = pd.concat([ids, ids_control])
    
    years = []
    months = []
    days = []
    for i in list(id_all.date_desinformacion):
        y, m, d = clean_dates(i)
        years.append(y)
        months.append(m)
        days.append(d)
        
    id_all['year'] = years
    id_all['month'] = months
    id_all['days'] = days
    return(id_all)

In [5]:
def clean_dates(x):
    if pd.isna(x):
        return(None, None, None)
    else:
        x = x.strftime('%Y-%m-%d')
        return(x.split('-')[0], 
              x.split('-')[1], 
              x.split('-')[2])

In [17]:
def download_media():
    media_id = pd.read_excel('3-synthetic_controls/newspapers/0-ids/media_ids.xlsx').drop(['Unnamed: 0'], axis = 1)
    media_id['date_desinformacion'] = media_id.apply(lambda x: datetime.date(int(x['year']), 
                                                                            int(x['month']), 
                                                                            int(x['days'])), axis = 1)
    return(media_id)

## Make Id Media Data

In [70]:
ids = obtain_id_newspapers()

In [71]:
articles_desinformacion = obtain_newspapers_id()

In [72]:
pd.merge(left = articles_desinformacion, 
        right = ids.drop(['link_desinformacion'], axis = 1), 
        on = 'id_desinformacion', 
        how = 'left').to_excel('3-synthetic_controls/newspapers/0-ids/media_ids.xlsx')

## Download controls per misinformation

In [ ]:
def match_media(df):
    for i in df.media_outlet.unique()

In [19]:
media_id = download_media()

In [20]:
for i in media_id.media_outlet.unique():
    pass

In [21]:
i

'codigooculto'